In [1]:
import numpy as np
import cv2


def bilinear_interpolate(_np_gray_image, _p1, _p2, _p3, _p4): # p1: ll, p2: lh, p3: hl, p4: hh
    x_max = max(_p1[0], _p2[0], _p3[0], _p4[0])
    y_max = max(_p1[1], _p2[1], _p3[1], _p4[1])
    n = 2
    ans = np.zeros((x_max, y_max , 2))
    for i in range(x_max * n):
        for j in range(y_max * n):
            u = i / (x_max * n)
            v = j / (y_max * n)
            x = round((1 - u) * ((1 - v) * _p1[0] + v * _p2[0]) + u * ((1 - v) * _p3[0] + v * _p4[0]))
            y = round((1 - u) * ((1 - v) * _p1[1] + v * _p2[1]) + u * ((1 - v) * _p3[1] + v * _p4[1]))
            try:
                ans[x][y][0] = (ans[x][y][0] * ans[x][y][1] + _np_gray_image[round(_np_gray_image.shape[0] * u)][round(_np_gray_image.shape[1] * v)]) / (ans[x][y][1] + 1)
                ans[x][y][1] += 1
            except:
                pass
    return ans[:, :, 0]

if __name__ == '__main__':
    image = cv2.imread(("test3.png"), cv2.IMREAD_GRAYSCALE)
    image = 255 - image
    img_y, img_x = image.shape
    print(img_x, img_y)

    a = 2000
    h = 130
    y = 39

    image = cv2.resize(image, dsize=(0, 0), fx=y / img_y, fy=y / img_y, interpolation=cv2.INTER_LINEAR)
    img_y, img_x = image.shape
    x = img_x
    Y = int(a * y / (h - y))
    X = int(x * (Y + a) / a)
    print(X, Y)
    print(img_x, img_y)

    p1 = (0, 0)
    p2 = (0,    X)
    p3 = (Y, X // 2 - img_x // 2)
    p4 = (Y, X // 2 + img_x // 2)
    name = bilinear_interpolate(image, p1, p2, p3, p4)
    _, name = cv2.threshold(name, 100, 255, cv2.THRESH_BINARY)
    name = cv2.GaussianBlur(name, (3,3), 0)
    _, name = cv2.threshold(name, 1, 255, cv2.THRESH_BINARY)
    cv2.imwrite("output3.png", name)

146 21
387 857
271 39


In [2]:
print(name[100])

[  0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
   0. 255. 255. 255. 255. 255. 255. 255. 255. 255. 255. 255. 255.   0.
   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0. 255.
 255. 255. 255. 255. 255. 255. 255.   0.   0.   0.   0.   0.   0.   0.
   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
   0.   0.   0.   0.   0.   0.   0.   0.   0.   0. 255. 255. 255. 255.
 255. 255. 255.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
   0.   0.   0.   0.   0.   0. 255. 255. 255. 255. 255. 255. 255.   0.
   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
   0.   0.   0.   0. 255. 255. 255. 255. 255. 255. 255. 255. 255. 255.
 255. 255. 255.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
   0. 

In [3]:
# serial protocol: 
# 1. All serial protocol consists of two independent strings.
# 2. First string is one of the following:
# - p -> push solenoid
# - P -> pull solenoid
# - r -> right mov
# - l -> left mov
# - d -> down mov
# - ` -> end sub serial bundle
# * But 'p' and 'P' hasn't number string. 
#
# 3. Second string is a integer number.
#
# * So, protocol has the following forms.
# ['p', `, 'r', '23', `, 'P', `, 'r', '2', `, 'd', '1', `, 'P', `, 'l', '23', `, ...]
#
#

import cv2
import numpy as np
from collections import deque

def conv_img_to_ser_deque(img):
    threshold_val = 255
    m, n = img.shape
    ans_deque = deque()
    col = 0
    for row in img:
        row_deque = deque(row)
        cnt = 0
        flag = -1
        if col % 2 == 0:
            while row_deque:
                tmp = row_deque.popleft()
                if tmp == 0 and flag == 0:
                    cnt += 1
                elif tmp == 0:
                    if flag != -1:
                        ans_deque.append(str(cnt))
                        ans_deque.append('`')
                    ans_deque.append('P')
                    ans_deque.append('`')
                    ans_deque.append('r')
                    cnt = 1
                    flag = 0
                elif tmp == threshold_val and flag == 1:
                    cnt += 1
                elif tmp == threshold_val:
                    if flag != -1:
                        ans_deque.append(str(cnt))
                        ans_deque.append('`')
                    ans_deque.append('p')
                    ans_deque.append('`')
                    ans_deque.append('r')
                    cnt = 1
                    flag = 1
        else:
            while row_deque:
                tmp = row_deque.pop()
                if tmp == 0 and flag == 0:
                    cnt += 1
                elif tmp == 0:
                    if flag != -1:
                        ans_deque.append(str(cnt))
                        ans_deque.append('`')
                    ans_deque.append('P')
                    ans_deque.append('`')
                    ans_deque.append('l')
                    cnt = 1
                    flag = 0
                elif tmp == threshold_val and flag == 1:
                    cnt += 1
                elif tmp == threshold_val:
                    if flag != -1:
                        ans_deque.append(str(cnt))
                        ans_deque.append('`')
                    ans_deque.append('p')
                    ans_deque.append('`')
                    ans_deque.append('l')
                    cnt = 1
                    flag = 1
        if cnt == n:
            ans_deque.pop()
            ans_deque.pop()
            ans_deque.append('`')
        else:
            ans_deque.append(str(cnt))
            ans_deque.append('`')
            col += 1
        ans_deque.append('d')
        ans_deque.append('1')
        ans_deque.append('`')
    return ans_deque

def conv_ser_deque_to_img(ser_deque, img_shape):
    threshold_val = 255
    ans_img = np.zeros(img_shape)
    x, y = 0, 0
    solenoid_state = 0
    while ser_deque:
        tmp = ser_deque.popleft()
        if tmp == 'd':
            y += int(ser_deque.popleft())
        elif tmp == 'p':
            solenoid_state = 1
            ser_deque.popleft()
        elif tmp == 'P':
            solenoid_state = 0
            ser_deque.popleft()
        elif tmp == 'r':
            r_mov_dis = int(ser_deque.popleft())
            _ = ser_deque.popleft()
            while r_mov_dis != 0:
                if solenoid_state == 1:
                    ans_img[y][x] = threshold_val
                x += 1
                r_mov_dis -= 1
        elif tmp == 'l':
            l_mov_dis = int(ser_deque.popleft())
            _ = ser_deque.popleft()
            while l_mov_dis != 0:
                x -= 1
                if solenoid_state == 1:
                    ans_img[y][x] = threshold_val
                l_mov_dis -= 1
    return ans_img

In [10]:
serial_deque = conv_img_to_ser_deque(name)